In [32]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [5]:
data=pd.read_csv("C:/Users/NITISH/Downloads/Book3555.csv")
data

,effects on health,reduce consumption,alternatives
0,0,0,Herbal tea
1,1,1,Water
2,1,1,Water
3,1,1,Water
4,1,1,Water
...,...,...,...
513,1,1,Coconut water
514,1,1,Water
515,1,0,Tea
516,1,1,Coffee


In [7]:
data.head()

,effects on health,reduce consumption,alternatives
0,0,0,Herbal tea
1,1,1,Water
2,1,1,Water
3,1,1,Water
4,1,1,Water


In [9]:
data.tail()

,effects on health,reduce consumption,alternatives
513,1,1,Coconut water
514,1,1,Water
515,1,0,Tea
516,1,1,Coffee
517,1,1,Herbal tea


In [11]:
data.info

<bound method DataFrame.info of      effects on health  reduce consumption  alternatives 
0                    0                   0     Herbal tea
1                    1                   1          Water
2                    1                   1          Water
3                    1                   1          Water
4                    1                   1          Water
..                 ...                 ...            ...
513                  1                   1  Coconut water
514                  1                   1          Water
515                  1                   0            Tea
516                  1                   1         Coffee
517                  1                   1     Herbal tea

[518 rows x 3 columns]>

Applying Logistics Regression to check whether people are concern about their health and reduce comsumption and what alternatives the took

In [36]:
# Convert 'alternatives' column into dummy variables
data_dummies = pd.get_dummies(data, columns=['alternatives '], drop_first=True)

In [72]:
# Convert boolean columns to integers
data_dummies = data_dummies.astype(int)

In [74]:
# Define the dependent and independent variables
X = data_dummies.drop('effects on health', axis=1)
y = data_dummies['effects on health']

In [76]:
# Add a constant to the independent variables
X = sm.add_constant(X)

In [78]:
# Check for perfect separation
crosstab = pd.crosstab(data['reduce consumption'], data['effects on health'])
print("Cross-tabulation of reduce_consumption with effects_on_health:")
print(crosstab)


Cross-tabulation of reduce_consumption with effects_on_health:
effects on health    0    1
reduce consumption         
0                   82   93
1                   29  314


In [80]:
# If perfect separation is detected, use penalized logistic regression
try:
    # Fit the logistic regression model
    model = sm.Logit(y, X)
    result = model.fit()
    summary = result.summary()
    print(summary)
except sm.tools.sm_exceptions.PerfectSeparationError:
    print("Perfect separation detected, using penalized logistic regression.")


Optimization terminated successfully.
         Current function value: 0.416068
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:      effects on health   No. Observations:                  518
Model:                          Logit   Df Residuals:                      509
Method:                           MLE   Df Model:                            8
Date:                Fri, 12 Jul 2024   Pseudo R-squ.:                  0.1992
Time:                        10:27:36   Log-Likelihood:                -215.52
converged:                       True   LL-Null:                       -269.14
Covariance Type:            nonrobust   LLR p-value:                 1.406e-19
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                         

In [84]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [86]:

# Fit a penalized logistic regression model
clf = LogisticRegression(penalty='l2', solver='liblinear')
clf.fit(X_scaled, y)

LogisticRegression(solver='liblinear')

In [90]:
# Display the coefficients
coef_df = pd.DataFrame({
        'Variable': X.columns,
        'Coefficient': clf.coef_[0]
    })
print(coef_df)

                                    Variable  Coefficient
0                                      const     0.000000
1                         reduce consumption     1.098199
2                       alternatives _Coffee    -0.113557
3                   alternatives _Herbal tea     0.110106
4           alternatives _Homemade smoothies    -0.211847
5  alternatives _Lemonade/limeade (homemade)    -0.149919
6    alternatives _Milk or milk alternatives    -0.036423
7                          alternatives _Tea     0.063428
8                        alternatives _Water    -0.125215


In [92]:
from sklearn.metrics import accuracy_score


In [98]:
# Predict the probabilities of the positive class
predicted_probs = result.predict(X)

In [100]:
# Convert probabilities to binary outcomes
predicted_classes = (predicted_probs >= 0.5).astype(int)

# Calculate the accuracy score
accuracy = accuracy_score(y, predicted_classes)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7895752895752896
